In [1]:
import datetime
import glob
import os

from arelle.ModelValue import qname
from arelle import Cntlr
import numpy as np

import pandas as pd
pd.reset_option('display.max_rows')
print(pd.options.display.max_rows)
pd.reset_option('display.max_columns')
print(pd.options.display.max_columns)
import plotly.graph_objects as go

import stock_research as sr
taiouhyou = sr.taiouhyou
import warnings
warnings.filterwarnings('ignore')

60
20


In [2]:
result_path = '../warehouse/stock/created/result_statistics/*'
files = glob.glob(result_path)
for file in files:
    print(file)
result_path = '../warehouse/stock/created/result_statistics\stock_statistics_20170301_20211201.csv'

../warehouse/stock/created/result_statistics\20220528_report.txt
../warehouse/stock/created/result_statistics\stock_statistics_20170301_20211201.csv


In [4]:
result = pd.read_csv(result_path)

In [23]:
today = datetime.datetime.today().strftime('%Y%m%d')
filepath = f'../warehouse/stock/created/result_statistics/{today}_report.txt'
th_opemargin = 40
th_sales_CAGR = 1.20
sentence = f'報告日 {today} 基準値\n営業CFマージン {th_opemargin}%\tCAGR {int(th_sales_CAGR*100)}%'+'\n'
sentence = today + sentence

In [37]:
with open(filepath, mode='a') as f:
    f.write(sentence)

res_dict = {}
for stock in result.secCode.unique():
    res_dict[f'{stock}'] = {}
    sdf = result.query(f'secCode=={stock}')
    filerName = sdf.filerName.reset_index(drop=True)[0]
    s = f'''{stock} {filerName}\n'''
    
    
    # 足切り 純利益が赤字
    
    # 営業キャッシュフローマージンが平均40%以上
    mean_1 = sdf.operatingincome_margin.replace('%','',regex=True).astype(float).mean()
    count_1 = sdf.operatingincome_margin.replace('%','',regex=True).astype(float).count()
    
    if mean_1 > th_opemargin:
        s1 = f'''{s} {count_1}年 {mean_1:.2f}% > {th_opemargin}%\n'''
        # print(s1)
        res_dict[f'{stock}']['opeCF']=[count_1,mean_1]
        # with open(filepath, mode='a') as f:
        #     f.write(s1)

    #############################################################
    # 連結会計
    tmp = sdf.query('NonConsolidated==0')
    tmp = tmp.query('element_id=="NetSalesSummaryOfBusinessResults"')
    
    if len(tmp)<2:
        continue
    
    # 幾何平均
    gmean_2 = (((tmp.Yoy.replace('%','',regex=True).astype(float)/100)[1:].cumprod(axis=0)[-1:]**(1/len((tmp.Yoy)[1:]))).values[0])
    count_2 = tmp.Yoy.replace('%','',regex=True).astype(float).count()
    if gmean_2 > th_sales_CAGR:
        s2 = f'''{s} CAGR({count_2}) {gmean_2:.3f}% > {th_sales_CAGR}%\n'''
        # print(s2)
        res_dict[f'{stock}']['CAGR']=[count_2,gmean_2]
        # with open(filepath, mode='a') as f:
        #     f.write(s2)
    #############################################################
    
    if (gmean_2 > th_sales_CAGR) and (mean_1 > th_opemargin):
        ss = f'''{s}CAGR({count_2}) > {gmean_2:.3f}\n営業CFマージン{th_opemargin} {count_1}年 {mean_1:.2f}% > {th_sales_CAGR}%\n'''
        with open(filepath, mode='a') as f:
            f.write(ss)
    
    

In [31]:
res_dict

{'9941': {'opeCF': [3, 47.333333333333336]},
 '4707': {},
 '7640': {},
 '4441': {},
 '3733': {'CAGR': [1, 1.23]},
 '7034': {},
 '2198': {},
 '7856': {},
 '8077': {},
 '2695': {'opeCF': [5, 60.8]},
 '3248': {},
 '1873': {'opeCF': [5, 41.8]},
 '9678': {},
 '7265': {'opeCF': [5, 72.6]},
 '5218': {'opeCF': [5, 53.2]},
 '1840': {},
 '3974': {},
 '3480': {'opeCF': [5, 63.4]},
 '3475': {'CAGR': [4, 1.3588475398583801]},
 '4880': {},
 '7983': {'opeCF': [5, 80.8]},
 '7831': {},
 '9824': {},
 '2345': {'CAGR': [2, 1.4629422408283932]},
 '4627': {'opeCF': [5, 72.4]},
 '7804': {},
 '3491': {'CAGR': [2, 1.469693845669907]},
 '9279': {'CAGR': [4, 1.2396216400777649]},
 '4378': {},
 '2375': {},
 '8079': {'opeCF': [5, 48.0]},
 '9251': {},
 '2373': {'opeCF': [5, 58.6]},
 '3804': {},
 '3195': {'opeCF': [2, 42.5]},
 '4355': {},
 '2315': {'opeCF': [2, 46.0]},
 '4666': {},
 '7131': {'opeCF': [2, 74.0]},
 '7378': {},
 '6309': {'opeCF': [5, 44.2]},
 '8917': {'opeCF': [3, 46.0]},
 '9603': {'opeCF': [3, 42.6666